#### source: https://github.com/google-research/bert/blob/master/README.md

### BERT fine-tune 實作分成三部分
### 1. 準備資料
### 2. 修改run_classifier.py 中的processor
### 3. 執行run_classifier.py 與設定參數

# ---------------------------------------------------

#### 1. 準備資料

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle
import uuid

In [2]:
df = pd.read_csv('./data/19999_question_category_a_v12.csv')

In [3]:
# df=df[((df.category_a_target != 7) & (df.category_a_target != 1) 
#        & (df.category_a_target != 2) & (df.category_a_target != 3))]

#df=df[((df.category_a_target != 7))]

In [4]:
df.head(2)

,description,category_a,category_b,category_c,category_d,category,description_clean,category_target
0,"無法會員登入,顯示訊息:無效的帳號或使用者不存在。帳號:96122401",CAMP,CAMP,CAMP,CAMP,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,因今天有事請假，要補登先前的加班，但開啟camp後，出勤表單顯示的內容看起來像測試的，沒有最...,CAMP,CAMP,CAMP,CAMP,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4


In [5]:
# t_df=df.loc[df['description']=='無法登入雲端']
# t_df

# t_df=df.loc[df['description']=='網管 Outlook一直Lag,無法作業。']
# t_df

In [6]:
df.shape

(8458, 8)

In [7]:
df.groupby(['category','category_target']).size()

category    category_target
Outlook與郵件  0                  1788
SAP         1                  1443
其他          2                  1853
網路          3                  1988
資訊系統        4                  1386
dtype: int64

In [8]:
#drop category
# df=df.loc[(df['category_target'] != 2) & (df['category_target'] != 7)]

In [9]:
df=df.drop(['category_a','category_b','category_c','category_d','description'], axis=1)
df.head(5)

,category,description_clean,category_target
0,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4
2,資訊系統,簽核BPM有問題 都顯示亂碼,4
3,資訊系統,無法在上面使用CAMP進行表單簽核動作,4
4,資訊系統,出差同仁無法登入camp 委請同事詢問,4


In [10]:
df.columns = ['category_a','description','category_a_target']

In [11]:
df.head(5)

,category_a,description,category_a_target
0,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4
2,資訊系統,簽核BPM有問題 都顯示亂碼,4
3,資訊系統,無法在上面使用CAMP進行表單簽核動作,4
4,資訊系統,出差同仁無法登入camp 委請同事詢問,4


In [12]:
#結果放到run_classifier.py get_labels 443行
sorted(df.category_a_target.unique())

[0, 1, 2, 3, 4]

In [13]:
pd.unique(df[['category_a', 'category_a_target']].values.ravel('K'))

array(['資訊系統', 'SAP', '網路', 'Outlook與郵件', '其他', 4, 1, 3, 0, 2],
      dtype=object)

In [14]:
pd.concat([df['category_a'], df['category_a_target']]).unique()

array(['資訊系統', 'SAP', '網路', 'Outlook與郵件', '其他', 4, 1, 3, 0, 2],
      dtype=object)

In [15]:
category_df =df[['category_a', 'category_a_target']]
category_df = category_df.drop_duplicates()
category_df

,category_a,category_a_target
0,資訊系統,4
14,SAP,1
524,網路,3
582,Outlook與郵件,0
6605,其他,2


In [16]:
df.shape

(8458, 3)

In [17]:
df.groupby(['category_a','category_a_target']).size()

category_a  category_a_target
Outlook與郵件  0                    1788
SAP         1                    1443
其他          2                    1853
網路          3                    1988
資訊系統        4                    1386
dtype: int64

### 現在我們將資料轉換成 BERT 需要的資料形式。根據官方fine tune教學，需要給的資料有四個column，分別是:
### (1) guid: Unique id for the example.

### (2) text_a: text_a: string. The untokenized text of the first sequence. For single sequence tasks, only this sequence must be specified.
### (3) text_b(Optional): string. The untokenized text of the second sequence. Only must be specified for sequence pair tasks.
### (4) label(Optional): string. The label of the example. This should be specified for train and dev examples, but not for test examples.

In [18]:
data_for_bert = []
strat_list = []
for i in range(len(df)):
    data_for_bert.append((i, #guid
                          df.iloc[i,:].description, #text_a
                          None, #text_b
                          df.iloc[i,:].category_a_target #label
                         ))
    strat_list.append((i,df.iloc[i,:].category_a_target))

In [19]:
len(data_for_bert)

8458

### 產生test.csv,下列兩項擇一

In [20]:
# #generate t_cest data from 19999_test_question_v1.csv
# test_df = pd.read_csv('./data/19999_test_question_v2.csv')
# test_df=test_df.drop(['category_a','category_a_b','description_clean'], axis=1)
# test_df
# test_df.columns = ['description','label']
# test_df.to_csv('tmp/call_center_input/test.csv', index=False, header=True)

In [21]:
#random select to test.csv
#import random
#test_df = pd.DataFrame(random.sample(data_for_bert, 10))
test_df = pd.DataFrame(data_for_bert)
test_df=test_df.sample(frac=0.05) #frac是要返回的比例
test_df.columns = ['id', 'description', 'text_b', 'label']
test_df_2=test_df.drop(['id', 'text_b'], axis=1)
#test_df
test_df_2.to_csv('tmp/call_center_input/test.csv', index=False, header=True)

### 產生test.tsv

In [22]:
#test.csv -> test.tsv
import pandas as pd
df_temp = pd.read_csv('tmp/call_center_input/test.csv') 
df_temp.to_csv('tmp/call_center_input/test.tsv', sep='\t', index=False, header=True) #

In [23]:
#test_df.head(5)
len(test_df)

423

In [24]:
len(data_for_bert)

8458

In [25]:
for s in test_df.id:
    data_for_bert=[k for k in data_for_bert if s not in k]

In [26]:
len(data_for_bert)

8035

In [27]:
for s in test_df.id:
    strat_list=[k for k in strat_list if s not in k]

In [28]:
len(strat_list)

8035

In [29]:
strat_list_temp = []
for s in strat_list:
    strat_list_temp.append(s[1])
strat_list=strat_list_temp

### 產生Train.tsv

In [30]:
train_df = pd.DataFrame(data_for_bert)
train_df.columns = ['id', 'description', 'text_b', 'label']
train_df_2=train_df.drop(['id', 'text_b'], axis=1)
#test_df
train_df_2.to_csv('tmp/call_center_input/train.csv', index=False, header=True)

In [31]:
#test.csv -> test.tsv
import pandas as pd
df_temp = pd.read_csv('tmp/call_center_input/train.csv') 
df_temp.to_csv('tmp/call_center_input/train.tsv', sep='\t', index=False, header=True) #

### 產生pickle_file

In [32]:
train, dev = train_test_split(data_for_bert, test_size=0.2, stratify=strat_list)

In [33]:
print('length for fine-tune training data: ',len(train))
print('length for fine-tune development data: ',len(dev))
print('first instance:', data_for_bert[0])

length for fine-tune training data:  6428
length for fine-tune development data:  1607
first instance: (0, '無法會員登入 顯示訊息 無效的帳號或不存在 帳號', None, 4)


In [34]:
with open('bert_train.p',mode = 'wb') as pickle_file:
    pickle.dump(train, pickle_file)

with open('bert_dev.p',mode = 'wb') as pickle_file:
    pickle.dump(dev, pickle_file)


#### 2. 於影片中說明如何修改 run_classifier.py 中的 processor

#### 3. 執行run_classifier.py 並設定參數

In [35]:
# import os
# os.environ["TF_CPP_MIN_LOG_LEVEL"]="3" 
#'1' # 這是默認的顯示等級，顯示所有信息
#'2' # 只顯示 warning 和 Error
#'3' # 只顯示 Error

In [36]:
!python run_classifier.py \
  --task_name=call_center \
  --do_train=true \
  --do_eval=true \
  --data_dir=. \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint= chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --output_dir=tmp/call_center_output/ \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_model_dir': 'tmp/call_center_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f73a1851128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=

### predict test.tsv

In [48]:
!python run_classifier.py \
  --task_name=call_center \
  --do_predict=true \
  --data_dir=tmp/call_center_input \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --output_dir=tmp/call_center_output/

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_model_dir': 'tmp/call_center_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7eff02dd3550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=

### 預測準確率

In [49]:
#test_df
#test_df2 = pd.read_csv('tmp/call_center_input/test.csv')
test_label = test_df['label'].tolist()
len(test_label)

423

In [50]:
test_predict_df = pd.read_csv('tmp/call_center_output/test_results.tsv',header=None,sep = '\t')
#test_predict_df

In [51]:
# a=np.argmax(test_predict_df.as_matrix(), axis=-1)  
# a

In [52]:
# #[5, 0, 4, 6]
# for n, i in enumerate(a):
#     if i == 0:
#         a[n] = 5
#     if i == 1:
#         a[n] = 0
#     if i == 2:
#         a[n] = 4  
#     if i == 3:
#         a[n] = 6
# a

In [53]:
test_predict_label =np.argmax(test_predict_df.as_matrix(), axis=-1)  
#test_predict_label =a

result = [1 if x==y else 0 for x,y in zip(test_label,test_predict_label)]


sum(result)/len(result)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.8132387706855791

In [54]:
# category_df[category_df['category_a_target']==8].category_a.iloc[0]

In [55]:
#test_predict_label[0]
predict_no = []
for i in range(len(test_predict_label)):
    if not (test_predict_label[i]==test_label[i]):
        predict_no.append((i,
                          #test_predict_label[i],
                          category_df[category_df['category_a_target']==test_predict_label[i]].category_a.iloc[0],
                          #test_label[i],
                          category_df[category_df['category_a_target']==test_label[i]].category_a.iloc[0],
                          test_df.iloc[i].description
                         ))
predict_no

[(25, '資訊系統', '網路', '無法進入BPM 系統  無法解析DNS  只能用IP 進入系統'),
 (26,
  '其他',
  'Outlook與郵件',
  '系統出現  此Microsoft office複本尚未啟動  您還剩27天可啟動 請確定您已連線至您的企業網路 已啟用自動啟動    請再'),
 (45,
  '網路',
  'Outlook與郵件',
  '1.點選mail中連結網址出現 這項作業因電腦中限制而取消  請系統管理員   2.連結公司網路Microsoft Lync 無法上線    連結私人網路可以'),
 (48, '網路', 'SAP', '網路正常 無法使用SAP 顯示無法連線10.243.22.59'),
 (57, '其他', '網路', '開機一段時間後 網路就出現異常訊息'),
 (59, '資訊系統', '網路', '需要上to be 課程 登入網頁 無法登入 流程需IE設定 需請系統管理員設定IE'),
 (66,
  '資訊系統',
  '其他',
  '出差已申權限 但無法 且顯示異常 有幾種情況 1. 沒有數字按鈕 2. 直接輸入號碼 無法按撥號 3. 連圖案都不見  系統為Lync2010'),
 (67, '資訊系統', 'SAP', '系統很慢  無法使用  ps.倉庫的也反應系統很慢'),
 (70, '其他', '網路', '廣達巴西提供的FTP server無法連結  帳號密碼無誤'),
 (71,
  '其他',
  'Outlook與郵件',
  '1. Office 產品金鑰遺失  2. 開啟公司薪資mail權限出現問題   3. 時間常停留會在2006年或亂跳 重設無用 開啟是在12/2時間   4. 右下角狀部份狀態圖示會不見 網路連線及附近icon    以上是時 電腦全錯亂時就發生問題'),
 (79, '網路', '其他', 'MYQUANTA無法登入'),
 (85, '資訊系統', '其他', '系統異常緩慢 開啟一個程式 就會整個停住'),
 (89,
  '網路',
  '其他',
  'Sir        但OCS 無法打回給客戶  還請協助 開通權限  出差時間 from 2/17 to 3/31   出差單號  APP90.'),
 (96, '其他', 

In [56]:
#t_df=df.loc[df['description']=='人在QCA出差 密碼過期無法收發Mail']

# t_df=df.loc[df['description']=='ProE 在使用  很容易被踢出  不確定是否是License 問題  或多人在使用']
# t_df

In [57]:
print(len(test_predict_label))
print(len(test_label))

423
423


In [47]:
# print(test_predict_label)
# print(test_label)